<img src="./img/hpe_logo.png" alt="HPE Logo" width="300">

<h1>Request Prediction from KServe InferenceService</h1>

<h5>Date: 11/8/23</h5>
<h5>Version: 2.0</h5>
<h5>Author(s): andrew.mendez@hpe.com</h5>



<img src="./img/platform_step0.png" alt="Enterprise Machine Learning platform architecture" width="850">

<h3>Import modules and define functions</h3>
The cell below imports all modules and libraries required to run the demo.

In [ ]:
# imports
import sys
import glob
import base64
import json
import requests
import matplotlib.pyplot as plt

from skimage import io
from PIL import Image, ImageDraw
from ipywidgets import interact, interactive
import ipywidgets as widgets
import io

from tqdm import tqdm
from multiprocessing import Pool

<h3>Step 1: Setting up connection details to KServe and define image directory</h3>

In [ ]:
images = r"./data/train_images_rgb_no_neg_filt_32/train_images_640_02_filt_32/*.jpg"
endpoint_name='detection-deploy'
model_name='sat-detection'
ingress_host = "{}.models.mlds-kserve.us.rdlabs.hpecorp.net".format(endpoint_name)
ingress_port = "80"
service_hostname = "{}.models.mlds-kserve.us.rdlabs.hpecorp.net".format(endpoint_name)
print(ingress_host)

In [ ]:
import torchvision
import torch
global is3classes
from collections import Counter
def run_nms(pred_d):
    d_cl_ids = {}
    n_classes = set([list(pred.keys())[0] for pred in pred_d])
    if 'Small Aircraft' in n_classes:
        cat2id = {'Fixed-wing Aircraft': 1, 'Cargo Plane':2,'Small Aircraft':3}
        cat2colors = {'Fixed-wing Aircraft': (255,0,0), 'Cargo Plane':(0,255,0),'Small Aircraft':(0,0,255)}
    else:
        cat2id = {'Fixed-wing Aircraft': 1, 'Cargo Plane':2}
    id2cat = {v:k for k,v in cat2id.items()}
    bboxes = []
    scores = []
    classes = []
    for pred in pred_d:
        cl_id = list(pred.keys())[0]
        classes.append(cat2id[cl_id])
        scores.append(pred['score'])
        bboxes.append(pred[cl_id])
    classes = torch.LongTensor(classes)
    bboxes = torch.FloatTensor(bboxes)
    scores = torch.FloatTensor(scores)
#     try:
#         min_value = torch.min(scores)
#         max_value = torch.max(scores)
#         percentile_50 = torch.quantile(scores, 0.5)
#         percentile_95 = torch.quantile(scores, 0.95)

#         # Convert tensors to numpy arrays for easy printing
#         min_value, max_value, percentile_50, percentile_95 = min_value.numpy(), max_value.numpy(), percentile_50.numpy(), percentile_95.numpy()

#         print(f"Min Conf: {min_value}")
#         print(f"Max Conf: {max_value}")
#         print(f"50th Percentile: {percentile_50}")
#         print(f"95th Percentile: {percentile_95}")
#     except Exception as e:
#         print(e)
#         pass
#     print(bboxes[:3])
#     idxs = scores>0.1

#     classes = classes[idxs]
#     bboxes = bboxes[idxs]
#     scores = scores[idxs]
    # idxs = torchvision.ops.batched_nms(bboxes, scores, classes, iou_threshold=0.2)
    idxs = torchvision.ops.nms(bboxes, scores, iou_threshold=0.2)

    final_classes = classes[idxs].tolist()
    final_bboxes = bboxes[idxs].tolist()
    final_scores = scores[idxs].tolist()
    # final_classes = classes.tolist()
    # final_bboxes = bboxes.tolist()
    # final_scores = scores.tolist()
    # print(final_scores[:3])
    final_d = []
    for cl,bbox,s in zip(final_classes,final_bboxes,final_scores):
        # print(s)
        final_d.append({id2cat[cl]:bbox, 'score':s })

    return final_d
def plot_pred(im,pred_d):
    '''
    '''
    draw = ImageDraw.Draw(im)
    cat2colors = {'Fixed-wing Aircraft': (255,0,0), 'Cargo Plane':(0,255,0),'Small Aircraft':(0,0,255)}
    cat2colors2 = {'Fixed-wing Aircraft': 'red', 'Cargo Plane':'green','Small Aircraft':'blue'}

    try:
        # calculate stats
        cl_names = []
        scores = []
        for pred in pred_d['predictions'][0]:
            assert len(list(pred.keys())) == 2
            cl_name = list(pred.keys())[0]
            bboxes = pred[cl_name]
            cl_names.append(cl_name)
            scores.append(pred['score'])
        # Use Counter to count frequencies
        frequency_counter = Counter(cl_names)
        # Count the number of unique values
        num_unique_values = len(frequency_counter)
        # Print the number of unique values
        print("{} Objects Detected:".format(len(cl_names)))
        print("{} Unique Classes Detected:".format(num_unique_values))
        # Print the unique values
        print("Unique Classes:", list(frequency_counter.keys()))
        # Print the frequency of each unique value
        for item, frequency in frequency_counter.items():
            print(f"{item} (color: {cat2colors2[item]}): {frequency}")
            
        for pred in pred_d['predictions'][0]:
            assert len(list(pred.keys())) == 2
            cl_name = list(pred.keys())[0]
            bboxes = pred[cl_name]
            # if pred['score'] > thres:
            # print("pred['score']: ",pred['score'])
            draw.rectangle([bboxes[0],bboxes[1],bboxes[2],bboxes[3]],outline=cat2colors[cl_name],fill=None,width=3)
            draw.text([bboxes[0],bboxes[1]-10],"{} :{:.2f}".format(cl_name,pred['score']),fill=(250,255,255))
        plt.figure(figsize=(8,8))
        
    except Exception as e:
        print(e)
        pass
    plt.imshow(im)
    plt.title("Model Predictions")
    plt.show()
    return im

def predict(args):
    '''
    Function to base64encode image and send to API
    '''
    image= Image.open(args[0])
    with io.BytesIO() as buffer:
        image.save(buffer, format='PNG')  # You can replace 'JPEG' with other formats like 'PNG' if needed
        image_bytes = buffer.getvalue()
    image_64_encode = base64.b64encode(image_bytes)
    bytes_array = image_64_encode.decode("utf-8")
    
    # Format the request in json
    request = {
      "instances":[
        {
          "data": bytes_array
        }
      ]
    }
    ingress_host = args[1]
    ingress_port = args[2]
    model_name = args[3]
    service_hostname = args[4]
    # Create request for Prediction (header, URL, payload)
    url = str("http://") + str(ingress_host) + ":" + str(ingress_port) + "/v1/models/" + str(model_name) + ":predict"
    headers = {'Host': service_hostname}
    payload = json.dumps(request)
    response = requests.post(url, data=payload, headers=headers)
    res = response.json()
    try:
        final_res = run_nms(res['predictions'][0])
        res['predictions'][0] = final_res

    except Exception as e:
        print(e)
        # final_res = res['predictions']
        pass
    
    return res

def visualize(idx):
    '''
    Visualize predicted results from resps
    '''
    # print(idx,thres)
    output = resps[idx]
    im = Image.open(imgs[idx])
    plot_pred(im,output)
    
def run_apply_async_multiprocessing(func, argument_list, num_processes):
    '''
    Use multiprocessing.apply_async to send simultaneous requests
    '''
    pool = Pool(processes=num_processes)

    jobs = [pool.apply_async(func=func, args=(*argument,)) if isinstance(argument, tuple) else pool.apply_async(func=func, args=(argument,)) for argument in argument_list]
    pool.close()
    result_list_tqdm = []
    for job in tqdm(jobs):
        result_list_tqdm.append(job.get())

    return result_list_tqdm

In [ ]:
imgs = [img for img in glob.glob(images, recursive=True)][:5]
LEN= len(imgs) # only get 5 images

<h3>Step 2: Request prediction from KServe InferenceService and display results</h3>

In [ ]:
resps = run_apply_async_multiprocessing(predict,[[imgs[i],ingress_host,ingress_port,model_name,service_hostname] for i in range(LEN)],num_processes=4)

In [ ]:
interact(visualize, idx=widgets.IntSlider(min=0, max=len(resps)-1, step=1, value=0));